In [1]:
import pysam
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import pyranges as pr

## Read in ref and non-ref insertions

In [2]:
# read the rmsk file
rmsk = pd.read_csv(
    snakemake.input.rmsk[0],
    skiprows=3,
    delim_whitespace=True,
    names=["Chromosome", "Start", "End", "Strand", "repeat"],
    usecols=[4, 5, 6, 8, 9],
)
# filter for rep_names
rep_names = [
    "L1HS_3end",
    "L1PA2_3end",
    "L1PA3_3end",
    "L1PA4_3end",
    "L1PA5_3end",
    "L1PA6_3end",
]
rmsk = rmsk[rmsk["repeat"].isin(rep_names)]
rmsk["Strand"] = rmsk.apply(lambda x: "+" if x.Strand == "+" else "-", axis=1)
rmsk = pr.PyRanges(rmsk)

In [3]:
knrgl = pd.read_csv(
    snakemake.input.knrgl[0],
    sep="\t",
    header=None,
    names=["Chromosome", "Start", "End", "Strand", "SVLEN", "SVTYPE"],
    dtype={"Chromosome": str, "Start": int, "End": int},
)
knrgl["Start"] = knrgl.apply(
    lambda x: x.Start - 750 if x.Strand == "-" else x.Start, axis=1
)
knrgl["End"] = knrgl.apply(lambda x: x.End + 750 if x.Strand == "+" else x.End, axis=1)
knrgl = pr.PyRanges(knrgl)

## Find peaks

In [4]:
class Peak:
    """Store collection of reads in a peak"""

    def __init__(self, r):
        self.chr = r.reference_name
        self.start = r.reference_start
        self.end = r.reference_end
        self.is_reverse = not r.is_reverse if r.is_read1 else r.is_reverse
        self.width = self.end - self.start
        if r.is_read1:
            self.r1 = [r]
            self.r1_ends = [r.query_alignment_end]
            self.r2 = []
            self.r2_starts = []
        elif r.is_read2:
            self.r1 = []
            self.r1_ends = []
            self.r2 = [r]
            self.r2_starts = [r.query_alignment_start]
        self.coverage = np.ones(r.reference_end - r.reference_start)
        assert self.end >= self.start, f"Peak end={self.end} < start={self.start}"

    def add_read(self, r):
        """Add read to peak, extending peak if necessary"""

        if r.is_read1:
            self.r1.append(r)
            self.r1_ends.append(r.query_alignment_end)
        elif r.is_read2:
            self.r2.append(r)
            self.r2_starts.append(r.query_alignment_start)

        # extend peak
        if r.reference_end > self.end:
            self.coverage = np.append(
                self.coverage, np.zeros(r.reference_end - self.end)
            )
            self.end = r.reference_end
            self.width = self.end - self.start

        start = r.reference_start - self.start
        end = r.reference_end - self.start
        self.coverage[start:end] += 1

        assert self.end >= self.start, f"Peak end={self.end} < start={self.start}"
        return self

    def merge(self, p):
        """Merge with another peak"""
        for r in p.r1:
            self.add_read(r)
        for r in p.r2:
            self.add_read(r)
            
        assert self.end >= self.start, f"Peak end={self.end} < start={self.start}"
        return self

In [1]:
def visualize_peaks(peaks):
	'''Visualize initial peak calls'''
	df = {
		"Chromosome": [],
		"Start": [],
		"End": [],
		"nreads": [],
		"width": [],
		"dist to next peak": [],
	}

	# iterate over peaks, collecting stats
	for i, p in enumerate(peaks):
		df["Chromosome"].append(p.chr)
		df["Start"].append(p.start)
		df["End"].append(p.end)
		df["nreads"].append(len(p.r1))
		df["width"].append(p.width)
		if i < len(peaks) - 1 and p.chr == peaks[i + 1].chr:
			df["dist to next peak"].append(peaks[i + 1].start - p.start)
		else:
			df["dist to next peak"].append(np.nan)

	# convert to dataframe
	df = pd.DataFrame.from_records(df).set_index(["Chromosome", "Start", "End"])

	# add rmsk and knrgl labels
	df_rmsk = pr.PyRanges(df.reset_index()).overlap(rmsk).df.set_index(["Chromosome", "Start", "End"])
	df_knrgl = pr.PyRanges(df.reset_index()).overlap(knrgl).df.set_index(["Chromosome", "Start", "End"])
	df["rmsk"] = df.index.isin(df_rmsk.index)
	df["knrgl"] = df.index.isin(df_knrgl.index)
	df["label"] = df.apply(lambda x: "rmsk" if x.rmsk else "knrgl" if x.knrgl else "other", axis=1)
	
	# plot
	fig = sns.JointGrid(data=df.reset_index(), x="width", y="nreads", hue="label", marginal_ticks=True)
	fig.plot_joint(sns.scatterplot, alpha = 0.5)
	fig.ax_joint.set(yscale = "log")

	fig.plot_marginals(sns.histplot, bins=100, element="step", fill=False)
	fig.ax_marg_x.set(yscale = "log")
	fig.ax_marg_y.set(xscale = "log")
	fig.ax_joint.set(xlabel = "Peak width (bp)", ylabel = "Number of reads")
	plt.show()
	
	fig = sns.histplot(data=df.reset_index(), x = "dist to next peak", bins=1000)
	fig.set(xscale = "log", xlabel = "Distance to next peak (bp)", ylabel = "Count", xlim=(1, max(df["dist to next peak"])))
	plt.show()

	return df

In [6]:
# STEP 1: make peaks from highest quality read1 alignments
bam = pysam.AlignmentFile(snakemake.input.bam, "rb")  # must be coordinate sorted
peaks = []; p = None
print("Making non-ref L1 peaks from high-quality R1 reads...")
for r in bam.fetch():

    # make peaks from read1 if
    # 1. is a read1 primary alignment
    # 2. mate aligns better to L1 than reference genome (YA >20, YA>YG)
    # 3. has high mapping quality (60), indicating it's uniquely mapped
    if (
        r.is_read1
        and (not r.is_unmapped)
        and (not r.is_secondary)
        and (not r.is_supplementary)
        and (r.has_tag("YA") and r.has_tag("YG"))
        and (r.get_tag("YA") > 20 and r.get_tag("YA") > r.get_tag("YG"))
        and (not r.has_tag("XA")) 	
        and (r.mapping_quality >= 60)
    ):

        # initialize
        if p is None:
            p = Peak(r)

        # if has mate in peak or overlap with peak, add to peak
        elif (
            (r.reference_name == p.chr)
            and (
                (r.is_read1 and (r.is_reverse != p.is_reverse))
                or (r.is_read2 and (r.is_reverse == p.is_reverse))
            )
            and (r.get_overlap(p.start, p.end) > r.query_alignment_length / 3)
        ):
            p.add_read(r)

        # else, make new peak
        else:
            peaks.append(p)
            p = Peak(r)
bam.close()

print(f"Found {len(peaks)} non-ref L1 peaks")
df = visualize_peaks(peaks)

Making non-ref L1 peaks from high-quality R1 reads...


In [8]:
# STEP 2: gather additional features for each peak
print("Gathering other reads at these peaks...")
bam = pysam.AlignmentFile(snakemake.input.bam, "rb")  
for p in peaks:
    for r in bam.fetch(p.chr, p.start, p.end):
        if not r.is_unmapped:
            p.add_read(r)

bam.close()
df = visualize_peaks(peaks)

Gathering other reads at these peaks...


In [9]:
%%time

# from https://github.com/oliviaguest/gini/blob/master/gini.py
def gini(array):
    """Calculate the Gini coefficient of a numpy array."""
    # based on bottom eq:
    # http://www.statsdirect.com/help/generatedimages/equations/equation154.svg
    # from:
    # http://www.statsdirect.com/help/default.htm#nonparametric_methods/gini.htm
    # All values are treated equally, arrays must be 1d:
    array = array.flatten()
    if np.amin(array) < 0:
        # Values cannot be negative:
        array -= np.amin(array)
    # Values cannot be 0:
    array = array + 0.0000001
    # Values must be sorted:
    array = np.sort(array)
    # Index per array element:
    index = np.arange(1,array.shape[0]+1)
    # Number of array elements:
    n = array.shape[0]
    # Gini coefficient:
    return ((np.sum((2 * index - n  - 1) * array)) / (n * np.sum(array)))

# STEP 3: merge peaks that are close together
print("Merging peaks that are close together...")
for i, p in enumerate(peaks):
	if i > 0 and p.chr == peaks[i-1].chr:
		if gini(p.coverage) < gini(p.merge(peaks[i-1]).coverage): # TODO save p.merge and use that, dont call merge twice
			p = p.merge(peaks[i-1])
			peaks[i-1] = None

new_peaks = []
for p in peaks:
	if p is not None:
		new_peaks.append(p)

print(f"{len(new_peaks)} remain after merging")
df = visualize_peaks(new_peaks)


76003